In [1]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017/")
db = client["stock_market"]
collection = db["stock_data"]

# Example: Get last 60 records for a ticker
def get_realtime_input(ticker, lookback=60):
    cursor = collection.find({"ticker": ticker}).sort("data.timestamp", -1).limit(lookback)
    records = []
    for doc in cursor:
        entry = doc["data"]
        entry["ticker"] = doc["ticker"]
        records.append(entry)
    df = pd.DataFrame(records).sort_values(by="timestamp")
    df.rename(columns={'Close': 'close'}, inplace=True)  # match with earlier LSTM code
    return df

In [11]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# To store prediction history

import os
import logging
import tensorflow as tf

# Suppress TensorFlow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Only show errors

# Suppress absl logging
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# Optional: Suppress tensorflow retracing warnings
logging.getLogger('tensorflow').setLevel(logging.ERROR)
tf.get_logger().setLevel(logging.ERROR)


prediction_history = []

def predict_realtime(ticker):
    model = load_model(f"saved models/saved models/{ticker}_lstm.h5", custom_objects={'mse': mse})
    scaler = joblib.load(f"saved models/saved models/{ticker}_scaler.pkl")

    realtime_df = get_realtime_input(ticker)
    if len(realtime_df) < 60:
        print("Not enough real-time data for prediction.")
        return

    X_input = prepare_input(realtime_df, scaler)[-1:]  # Last lookback window
    y_pred = model.predict(X_input)

    padded_pred = np.hstack([y_pred, np.zeros_like(y_pred)])
    predicted_price = scaler.inverse_transform(padded_pred)[0][0]

    # Get actual recent close price
    actual_price = realtime_df['close'].iloc[-1]
    timestamp = realtime_df['timestamp'].iloc[-1]

    prediction_history.append({
        'timestamp': timestamp,
        'actual': actual_price,
        'predicted': predicted_price
    })

    print(f"📊 [{timestamp}] Predicted next close price for {ticker}: ${predicted_price:.2f} | Actual: ${actual_price:.2f}")

    # Plot predictions
    # if len(prediction_history) > 1:
    #     df_history = pd.DataFrame(prediction_history)
    #     plt.figure(figsize=(10, 4))
    #     plt.plot(df_history['timestamp'], df_history['actual'], label='Actual Price', marker='o')
    #     plt.plot(df_history['timestamp'], df_history['predicted'], label='Predicted Price', marker='x')
    #     plt.xticks(rotation=45)
    #     plt.title(f"{ticker} - Real vs Predicted Close Price")
    #     plt.xlabel("Timestamp")
    #     plt.ylabel("Price ($)")
    #     plt.legend()
    #     plt.tight_layout()
    #     plt.grid(True)
    #     plt.show()


In [13]:
predict_realtime("AAPL")
predict_realtime('ACN')
predict_realtime('GOOGL')
predict_realtime('MSFT')
predict_realtime('AMD')
predict_realtime('AMZN')
predict_realtime('IBM')
predict_realtime('ORCL')
predict_realtime('INFY')
predict_realtime('SAP')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for AAPL: $197.20 | Actual: $196.87
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for ACN: $256.25 | Actual: $282.40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for GOOGL: $207.90 | Actual: $151.21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for MSFT: $333.40 | Actual: $367.60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for AMD: $84.11 | Actual: $87.43
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for AMZN: $195.55 | Actual: $172.59
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for IBM: $231.58 | Actual: $238.81
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
📊 [2025-04-20 22:25:37] Predicted next close price for ORCL: $123.97 | Actual: $128